In [71]:
import gurobipy as gp
import pandas as pd
from gurobipy import GRB

def optimize_preferences(data_path, epsilon=0.00001):
    #Charger et trier les alternatives
    df_preferences = pd.read_excel(data_path, index_col=0)
    df_preferences = df_preferences.sort_index() 
    preferences_id = df_preferences.index.to_list()

    #Nombre de points de discrétisation pour chaque critère
    points = {1: 4, 
                             2: 4, 
                             3: 4}

    valeurs_min = {
        1: df_preferences["distance"].min(),
        2: df_preferences["max_workload"].min(),
        3: df_preferences["#changed offices"].min(),
    }

    valeurs_max = {
        1: df_preferences["distance"].max(),
        2: df_preferences["max_workload"].max(),
        3: df_preferences["#changed offices"].max(),
    }

    #Initialisation du modèle d'optimisation
    modele = gp.Model("Optimisation_Preferences")

    #Variables de décision
    fonctions_utilite = {
        critere: modele.addVars(points[critere] + 1, name=f"u_{critere}")
        for critere in range(1,4)
    }
    
    ecart_positif = modele.addVars(preferences_id, name="ecart_positif")
    ecart_negatif = modele.addVars(preferences_id, name="ecart_negatif")

    #Fonction d'utilité partielle pour un critère donné
    def utilite_partielle(critere, valeur_x):
        if valeur_x <= valeurs_min[critere]:
            return fonctions_utilite[critere][0]
        if valeur_x >= valeurs_max[critere]:
            return fonctions_utilite[critere][points[critere]]

        #Trouver l'intervalle correspondant
        index_k = int(points[critere] * (valeur_x - valeurs_min[critere]) /
                      (valeurs_max[critere] - valeurs_min[critere]))
        valeur_k = valeurs_min[critere] + (valeurs_max[critere] - valeurs_min[critere]) * index_k / points[critere]

        #Interpolation linéaire
        return fonctions_utilite[critere][index_k] + \
               (valeur_x - valeur_k) / (valeurs_max[critere] - valeurs_min[critere]) * (
                   fonctions_utilite[critere][index_k + 1] - fonctions_utilite[critere][index_k]
               )

    #Fonction d'utilité globale
    def utilite_globale(alternative):
        return sum(utilite_partielle(critere, alternative[critere]) for critere in range(1,4))

    #Fonction objectif : minimisation de l'erreur d'ajustement
    somme_ecarts = gp.quicksum(ecart_positif[a] + ecart_negatif[a] for a in preferences_id)
    modele.setObjective(somme_ecarts, GRB.MINIMIZE)

    #Contraintes sur les fonctions d'utilité partielle (croissantes)
    for critere in range(1,4):
        for index in range(points[critere]):
            modele.addConstr(fonctions_utilite[critere][index] + epsilon <= fonctions_utilite[critere][index + 1])

    #Normalisation : l'utilité minimale est 0 et la somme des max vaut 1
    for critere in range(1,4):
        modele.addConstr(fonctions_utilite[critere][0] == 0)
    modele.addConstr(gp.quicksum(fonctions_utilite[critere][points[critere]] for critere in range(1,4)) == 1)

    #Contraintes de respect des préférences
    for index in range(len(preferences_id) - 1):
        alt_actuelle = {
            1: df_preferences.loc[preferences_id[index], "distance"],
            2: df_preferences.loc[preferences_id[index], "max_workload"],
            3: df_preferences.loc[preferences_id[index], "#changed offices"],
        }
        alt_suivante = {
            1: df_preferences.loc[preferences_id[index + 1], "distance"],
            2: df_preferences.loc[preferences_id[index + 1], "max_workload"],
            3: df_preferences.loc[preferences_id[index + 1], "#changed offices"],
        }

        modele.addConstr(
            utilite_globale(alt_actuelle) - ecart_positif[preferences_id[index]] + ecart_negatif[preferences_id[index]] + epsilon
            <= utilite_globale(alt_suivante) - ecart_positif[preferences_id[index + 1]] + ecart_negatif[preferences_id[index + 1]]
        )

    #Résolution du modèle
    modele.params.LogToConsole = 0
    modele.optimize()

    print("Erreur d'ajustement:", modele.objVal)

    #Calcul et ajout de l'utilité globale aux alternatives
    df_preferences["utilite_globale"] = [
        utilite_globale({1: x[0], 2: x[1], 3: x[2]}).getValue() for x in df_preferences.values
    ]

    return df_preferences

#1ère Utilisation avec fichier preferences.xlsx:
resultats = optimize_preferences("./data/Preferences.xlsx")
print(resultats)

Set parameter LogToConsole to value 0
Erreur d'ajustement: 0.2439562975176628
        distance  max_workload  #changed offices  utilite_globale
rank                                                             
1      83.722254      1.066358                 1         0.032654
2      86.064870      1.123243                 0         0.048427
3      89.663945      1.114196                 1         0.057960
4      80.186376      1.226844                 1         0.185964
5      79.227614      1.180500                 2         0.278463
6      75.214893      1.404830                 0         0.278473
7      83.308010      1.471083                 0         0.297105
8      82.397314      1.409648                 1         0.297115
9      81.466460      1.029212                 4         0.310724
10     91.784421      1.087131                 3         0.255943
11    103.318696      1.223812                 0         0.346500
12     94.123879      1.306550                 0         0.28343